In [5]:
import os

import pandas as pd

In [2]:
if not os.path.exists('python'):
    os.chdir('..')

In [3]:
def read_word_counts():
    with open('words_counts.txt', 'r') as f:
        return f.read()

In [57]:
def create_df():
    word_counts_text = read_word_counts()
    words_counts_list = []
    for words_count_line in word_counts_text.strip().split('\n'):
        word, count = words_count_line.split(' ')
        words_counts_list.append({
            'word': word,
            'count': int(count)
        })
    return pd.DataFrame(words_counts_list)
df = create_df()

In [58]:
from collections import Counter

In [59]:
# import re
# re.sub('[^a-z]*', '', 'kj _asd 123ksdf')

In [60]:
c = Counter()
for word in df.word:
    c.update(word)
#     break

In [61]:
# c.update('kj')

In [62]:
c

Counter({'w': 969,
         'h': 1877,
         'i': 4100,
         'c': 2144,
         't': 3305,
         'e': 6035,
         'r': 4126,
         'a': 6916,
         'b': 1701,
         'o': 4114,
         'u': 2774,
         'l': 3442,
         'd': 2228,
         'm': 1972,
         'v': 668,
         's': 4391,
         'f': 972,
         'g': 1580,
         'n': 3316,
         'y': 2042,
         'k': 1418,
         'p': 1926,
         'q': 116,
         'j': 294,
         'z': 389,
         'x': 340})

In [51]:
re.split('[^a-z]*', '')

['']

In [56]:
re.search('^[a-z]+$', 'wat_ch')